In [56]:
from test_utils import get_feature, load_model
from tqdm import tqdm
from deepface import DeepFace
import random
import os
import cv2
import numpy as np
import torch
from PIL import Image
from glob import glob
import matplotlib.pyplot as plt



In [57]:
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    
    similarity = dot_product / (norm_vector1 * norm_vector2)

    return similarity

In [58]:

def l2_distance(vector1, vector2):
    distance = np.linalg.norm(vector1 - vector2)
    return distance

In [59]:
dataset_path = "../lfw/"

In [60]:
def detectFace(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    face_objs = DeepFace.extract_faces(img_path = img, 
                                            target_size = (112, 112), 
                                            detector_backend = 'yunet', 
                                            enforce_detection=False)
    face = face_objs[0]['face']
    bbox = face_objs[0]['facial_area']
    return face, bbox


In [61]:
model = load_model()

In [62]:
def test_frr(dataset_path, threshold):
    false_refusal = 0
    total = 0
    frr_score = []
    for person in tqdm(os.listdir(dataset_path)):
        person_path = os.path.join(dataset_path, person)
        images = os.listdir(person_path)
        img1 = cv2.imread(os.path.join(person_path, images[0]))
        face1, bbox1 = detectFace(img1)
        feature1 = get_feature(model, face1)
        for image in images[1:]:
            img2 = cv2.imread(os.path.join(person_path, image))
            face2, bbox2 = detectFace(img2)
            feature2 = get_feature(model, face2)
            dist = cosine_similarity(feature1, feature2)
            frr_score.append(dist)
            if dist < threshold:
                false_refusal += 1
            total += 1
    frr = false_refusal / total
    return frr, frr_score
        


In [63]:
def test_far(dataset_path, threshold):
    false_acceptance = 0
    total = 0
    far_score = []
    for person in tqdm(os.listdir(dataset_path)):
        person_path = os.path.join(dataset_path, person)
    
        images = os.listdir(person_path)
        img1 = cv2.imread(os.path.join(person_path, images[0]))
        face1, bbox1 = detectFace(img1)
        feature1 = get_feature(model, face1)

        other_images = glob(os.path.join(dataset_path, "*/*"))
        # get random 5 images from other people
        five_images = random.sample(other_images, 5)
        for image in five_images:
            if person == image.split("/")[-2]:
                continue
            img2 = cv2.imread(image)
            face2, bbox2 = detectFace(img2)
            feature2 = get_feature(model, face2)
            dist = cosine_similarity(feature1, feature2)
            far_score.append(dist)
            if dist > threshold:
                false_acceptance += 1
            total += 1
    far = false_acceptance / total
    return far, far_score

In [66]:
mps_device = torch.device("mps")
mps_device

device(type='mps')

In [67]:
frr, frr_score = test_frr(dataset_path, 0.7)

100%|██████████| 5749/5749 [15:58<00:00,  6.00it/s]  


In [68]:
far, far_score = test_far(dataset_path, 0.7)

100%|██████████| 5749/5749 [57:15<00:00,  1.67it/s]


In [69]:
print("FRR: ", frr)
print("FAR: ", far)

FRR:  0.9485569214323891
FAR:  0.001774900814366256


In [ ]:
# from sklearn import metrics
# def optimal_threshold(frr_score, far_score):
#     Y_true = [1]*len(frr_score) + [0]*len(far_score)
#     Y_score = frr_score + far_score
#     assert len(Y_true) == len(Y_score)
#     fpr, tpr, thresholds = metrics.roc_curve(Y_true,Y_score)
#     threshold = thresholds[np.argmin(abs(tpr-(1-fpr)))]
#     return threshold

In [70]:
from sklearn.metrics import roc_curve

def find_threshold(inData, outData, in_low = True):
    allData = np.concatenate((inData, outData))
    labels = np.concatenate((np.zeros(len(inData)), np.ones(len(outData))))
    fpr, tpr, thresholds = roc_curve(labels, allData, pos_label = in_low)
    distances = np.sqrt(np.square(1 - tpr) + np.square(fpr))
    best_index = np.argmin(distances)

    optimal_threshold = thresholds[best_index]
    return optimal_threshold

In [71]:
optimal_threshold = find_threshold(far_score, frr_score)
optimal_threshold

0.25190312

In [72]:
frr, _ = test_frr(dataset_path, optimal_threshold)
print("FRR: ", frr)

far, _ = test_far(dataset_path, optimal_threshold)
print("FAR: ", far)


100%|██████████| 5749/5749 [15:34<00:00,  6.15it/s]  


FRR:  0.36878674505611975


100%|██████████| 5749/5749 [56:48<00:00,  1.69it/s]

FAR:  0.22522616562282533


In [73]:
print("FRR: ", frr)
print("FAR: ", far)

FRR:  0.36878674505611975
FAR:  0.22522616562282533


In [ ]:
img1 = cv2.imread('../faceData/all/30013/trung le tuan anh2.png')
face1, bbox1 = detectFace(img1)
feature1 = get_feature(model, face1)
print(feature1)

[-0.02875417  0.18302968 -0.02310282  0.08957249 -0.07080924  0.10270294
  0.07103153 -0.0265419   0.14708355  0.02778193  0.06034835  0.0718666
  0.05690104 -0.18799436 -0.11775121 -0.09148578 -0.19917274 -0.08114896
  0.17435133  0.05830747 -0.04905382  0.06347769  0.17613931  0.09205694
 -0.04880379 -0.04811094  0.01275891 -0.11068427  0.05892752 -0.03917056
 -0.02844682 -0.11119367 -0.02415268  0.03327522 -0.07994351 -0.01618305
 -0.09409409  0.0984084   0.1266001   0.02317806  0.02782397  0.00887924
 -0.01814869  0.06260785 -0.07531265 -0.00574741 -0.04088124 -0.01128512
  0.0619637  -0.0173854   0.10024048 -0.00559668  0.05669412 -0.01407709
  0.00426094 -0.09513994  0.01949965 -0.17669638  0.01132592 -0.14847592
  0.01268391 -0.13354065  0.11224999  0.03254943  0.03158768 -0.09911004
 -0.0105359   0.023438    0.07798124 -0.07104673  0.16351186  0.08950756
  0.02084881 -0.04554148 -0.09403953  0.0675594   0.04337565  0.11393401
 -0.07515387  0.13770904 -0.07212098  0.03692792  0.

In [ ]:
img2 = cv2.imread('../faceData/all/10003/2022_10_18_06_40_43__2.jpg')
face2, bbox2 = detectFace(img2)
feature2 = get_feature(model, face2)
print(feature2)

[-0.02540819  0.08232027  0.13514179  0.03857974  0.00129554  0.17379831
  0.06088528 -0.04762411  0.0067329  -0.09327266 -0.0433346   0.13426739
  0.04556261 -0.1798195  -0.01627577 -0.01622049  0.03196425  0.01007559
  0.04106614 -0.11299579  0.08861656  0.02189608  0.06434159  0.08054918
 -0.07806543  0.12591861 -0.06922673 -0.07399234  0.06061123 -0.04016552
 -0.0736222   0.01006695 -0.11398298 -0.08657749  0.0698527  -0.01126333
 -0.08418511  0.02214006  0.07320255  0.02537812 -0.00859671 -0.17972736
 -0.02086715  0.05421015  0.09071898  0.07568161  0.19339558  0.11683965
  0.12500294  0.06061844  0.10713025  0.15333855 -0.07431196  0.04304335
 -0.02875019  0.00033819 -0.01615198 -0.15406917  0.07553797  0.11244022
  0.05585319  0.01246117  0.05475994 -0.14491051  0.04159438 -0.03290727
 -0.08409759  0.28009358 -0.01369365 -0.06119802  0.05615874  0.11622646
  0.04642806  0.08633889  0.01055305  0.14303547  0.02425816  0.16851835
 -0.03621553 -0.05905079  0.08392548 -0.02157014 -0

In [ ]:
cos = cosine_similarity(feature1, feature2)
cos

0.25172397